In [1]:
# Import Libraries
import requests
import json
import time

In [2]:
hdr = {'User-Agent': 'windows:r/ethereum.single.result:v1.0' +
       '(by /u/Budabellly)'}
url = 'https://www.reddit.com/r/ethereum/.json'
req = requests.get(url, headers=hdr)
json_data = json.loads(req.text)


In [ ]:
# # Example of request
# posts = json.dumps(json_data['data']['children'], indent=4, sort_keys=True)
# print(posts)

In [4]:
data_all = json_data['data']['children']
num_of_posts = 0
while len(data_all) <= 100:
    time.sleep(2)
    last = data_all[-1]['data']['name']
    url = 'https://www.reddit.com/r/ethereum/.json?after=' + str(last)
    req = requests.get(url, headers=hdr)
    data = json.loads(req.text)
    data_all += data['data']['children']
    if num_of_posts == len(data_all):
        break
    else:
        num_of_posts = len(data_all)

KeyboardInterrupt: 

In [6]:
print json.dumps(data_all, indent=4, sort_keys=True)

[
    {
        "data": {
            "approved_at_utc": null, 
            "approved_by": null, 
            "archived": false, 
            "author": "5chdn", 
            "author_flair_css_class": null, 
            "author_flair_text": " \u2001\u2001\u2001\u2001Flair\u2001\u2001\u2001\u2001", 
            "banned_at_utc": null, 
            "banned_by": null, 
            "brand_safe": true, 
            "can_gild": false, 
            "can_mod_post": false, 
            "clicked": false, 
            "contest_mode": false, 
            "created": 1508472777.0, 
            "created_utc": 1508443977.0, 
            "distinguished": null, 
            "domain": "self.ethereum", 
            "downs": 0, 
            "edited": 1508446021.0, 
            "gilded": 0, 
            "hidden": false, 
            "hide_score": false, 
            "id": "77gytn", 
            "is_crosspostable": false, 
            "is_reddit_media_domain": false, 
            "is_self": true, 
            